## fastcc

In [ ]:
__author__ = "Nantia Leonidou"
__description__ = " Main script "

import pandas as pd
import numpy as np
from rank.rank_reactions import *
from prune.prune_model import *

In [ ]:
# read model
model = io.read_sbml_model('../pre_processing/dataset/recon1_with_BOF_and_VBOF.xml')
# genes
G = pd.read_csv('../pre_processing/dataset/1_GPL570_GSE3397/1_GPL570_GSE3397_entrez_ids.csv')
G = list(G['ENTREZ_GENE_ID'])
# ubiquity scores
U = pd.read_csv('../pre_processing/dataset/1_GPL570_GSE3397/1_GPL570_GSE3397_ubiquity.csv', header=None)
U = U.rename(columns={0: "Scores"})
U = list(U['Scores'])
#confidence_scores
#confidence_scores = get_test_inputs('../testInputs.mat','../humanModel.mat')[3]
#confidence_scores[-1] = 4
confidence_scores = pd.read_csv('../pre_processing/dataset/Recon1_confidence_scores_with_BOF_and_VBOF.csv')
confidence_scores = np.float64(list(confidence_scores['Confidence Score']))

In [ ]:
GM, C, NC, P, Z, model_C = rank_reactions(model, G, U, confidence_scores, [], 2)   

In [ ]:
print(len(GM.reactions))
print(len(GM.metabolites))
print(len(GM.genes))
print(len(C))
print(len(NC))
print(len(P))
print(len(Z))
print(len(model_C))

In [ ]:
print('Generic model passed precursor metabolites test')

##############################################################
# If generic functionality test is passed, prune reactions
###############################################################
try:
    
    print('Pruning reactions...')
    t0 = process_time()
    PM, cRes = prune_model(GM, P, C, Z, 1/3, '../pre_processing/dataset/key_metabolites_RECON1.txt', 1, 2)
    # Stop the stopwatch / counter
    t_stop = process_time()
    # compute elapsed time
    pruneTime = t_stop - t0
    
    print('PASS...', 'Function prune_model ran without error')

except (RuntimeError, TypeError, NameError) as inst:

    print('FAIL...', 'Function prune_model was terminated with the error:')
    print(type(inst))
    print(inst.args)

In [ ]:
print(len(PM.reactions))
print(len(PM.metabolites))
print(len(PM.genes))
print(len(cRes))
print(pruneTime)

In [ ]:
hours, rem = divmod(t_stop-t0, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [ ]:
io.write_sbml_model(PM,"final_python_recon1_with_BOF_BEC1_method2.xml")